In [1]:
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re

In [2]:
# read file
f = open("./weibos.txt", 'r', encoding="UTF-8")
text = f.read()

In [3]:
text

'#斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用。 \u200b\n国足输给叙利亚之后，里皮辞职。谁将成为新主帅，成为广大球迷关注的焦点。目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里。 \u200b\n据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决。据了解。比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战。辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利。这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话。至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商。\n让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢……。来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行\n斯科拉里想执教中国国足！老头有点意思，凡是里皮干过的地方，他就想试试。当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛！可以试试！\n斯科拉里的水平，还不如里皮。斯科拉里，看好的不是国足，而是年薪…… \u200b\n非常应该辞职！中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有！从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了！穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪\n赛后，叙利亚主教练在更衣室里给每个队员一个耳光。主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队！中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去！”，球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛？”\n里皮辞职返回意大利，他的助教马达洛尼随队返回广州。马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望他在考虑一下，我也建议他重新考虑，但无济于事。”\n中国足协：接受里皮辞职请求，将深刻反思\n看了个报道，马达洛尼说：“关于里皮的辞职，我事先也没有被告知，自己也

In [4]:
# prepocessing text using regular expression
sentences = re.split('[。！？]', text.replace('\n', '')) # split with 。!? and replace \n

In [5]:
sentences

['#斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用',
 ' \u200b国足输给叙利亚之后，里皮辞职',
 '谁将成为新主帅，成为广大球迷关注的焦点',
 '目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里',
 ' \u200b据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决',
 '据了解',
 '比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战',
 '辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利',
 '这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话',
 '至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商',
 '让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢……',
 '来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行斯科拉里想执教中国国足',
 '老头有点意思，凡是里皮干过的地方，他就想试试',
 '当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛',
 '可以试试',
 '斯科拉里的水平，还不如里皮',
 '斯科拉里，看好的不是国足，而是年薪…… \u200b非常应该辞职',
 '中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有',
 '从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了',
 '穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪赛后，叙利亚主教练在更衣室里给每个队员一个耳光',
 '主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队',
 '中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去',
 '”，球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛',
 '”里皮辞职返回意大利，他的助教马达洛尼随队返回广州',
 '马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望

In [6]:
if sentences[len(sentences)-1] == '':
    sentences.pop()

In [7]:
# read the stop words
stop = [line.strip().decode('utf-8') for line in open('stopword.txt').readlines()]

In [8]:
# segment words form sentence function
def get_item_str(item_text):
    item_str = ""
    item=(pseg.cut(item_text))
    for e in list(item):
        if e.word not in list(stop):
            item_str += e.word
            item_str += " "
    return item_str


In [9]:
# define min_hash funciton
def get_minhash(item_str):
    temp = MinHash()
    for e in item_str:
        temp.update(e.encode('utf8'))
    return temp

In [10]:
# get the documents after segment words
documents = []
for item_text in sentences:
    item_str = get_item_str(item_text)
    documents.append(item_str)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Dumping model to file cache /tmp/jieba.cache
Dump cache file failed.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/jieba/__init__.py", line 154, in initialize
    _replace_file(fpath, cache_file)
PermissionError: [Errno 1] Operation not permitted: '/tmp/tmpzjybybnl' -> '/tmp/jieba.cache'
Loading model cost 1.659 seconds.
Prefix dict has been built successfully.


In [11]:
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    # get train_documents[i] MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)

In [12]:
forest.index()

In [13]:
query = '足协主席陈戌源看望国足：力争昂首挺胸打进世界杯'
item_str = get_item_str(query)
minhash_query = get_minhash(item_str)

In [14]:
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
print("Top 3 邻居", result)

0 0.2109375 #斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用
29 0.203125 ​带队结果绝不会比里皮更差，​而且我一定能带国足夺得2022世界杯冠军
31 0.1015625 ​曾经也以为世界名帅里皮能够带领国足走出迷茫，让国足蒸蒸日上，也让我们国人不再迷茫，吃下一颗定心丸
Top 3 邻居 [0, 29, 31]
